In [ ]:
"""
Notebook 03: Bayesian Change Point Detection
=============================================
This notebook implements Bayesian change point detection using PyMC to identify
structural breaks in Brent oil price log returns.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append(str(Path().resolve().parent / "src"))
from data_loader import load_brent_data, calculate_returns

# Load data
print("Loading data...")
df = load_brent_data()
df['log_return'] = calculate_returns(df, method='log')

# Prepare returns for modeling
returns = df['log_return'].dropna().values
returns_dates = df['log_return'].dropna().index
n = len(returns)

print(f"Data prepared: {n} observations")
print(f"Date range: {returns_dates.min()} to {returns_dates.max()}")
print(f"\nReturns statistics:")
print(f"  Mean: {returns.mean():.6f}")
print(f"  Std: {returns.std():.6f}")
print(f"  Min: {returns.min():.6f}")
print(f"  Max: {returns.max():.6f}")

# Build Bayesian Change Point Model
print("\n" + "=" * 60)
print("BUILDING BAYESIAN CHANGE POINT MODEL")
print("=" * 60)

with pm.Model() as change_point_model:
    # Prior for change point location (discrete uniform over all time points)
    tau = pm.DiscreteUniform("tau", lower=1, upper=n-1)
    
    # Priors for mean returns before and after change point
    mu_1 = pm.Normal("mu_1", mu=0, sigma=0.1)  # Before change point
    mu_2 = pm.Normal("mu_2", mu=0, sigma=0.1)   # After change point
    
    # Prior for standard deviation (shared across regimes)
    sigma = pm.HalfNormal("sigma", sigma=0.1)
    
    # Switch function: use mu_1 before tau, mu_2 after tau
    mu = pm.math.switch(tau > np.arange(n), mu_1, mu_2)
    
    # Likelihood
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=returns)
    
    print("\nModel defined. Starting MCMC sampling...")
    print("This may take several minutes...")
    
    # Sample from posterior
    trace = pm.sample(
        draws=2000,
        tune=1000,
        return_inferencedata=True,
        random_seed=42,
        progressbar=True
    )

print("\nSampling complete!")


In [ ]:
# Model Diagnostics
print("\n" + "=" * 60)
print("MODEL DIAGNOSTICS")
print("=" * 60)

# Summary statistics
summary = az.summary(trace, var_names=["tau", "mu_1", "mu_2", "sigma"])
print("\nPosterior Summary Statistics:")
print(summary)

# Check convergence (R-hat should be close to 1.0)
print("\nConvergence Check (R-hat):")
rhat_values = summary['r_hat']
for var, rhat in rhat_values.items():
    status = "✓ Good" if 0.99 < rhat < 1.01 else "⚠ Warning"
    print(f"  {var}: {rhat:.4f} {status}")

# Trace plots
print("\nGenerating trace plots...")
az.plot_trace(trace, var_names=["tau", "mu_1", "mu_2", "sigma"], 
               compact=True, figsize=(12, 10))
plt.tight_layout()
plt.show()


In [ ]:
# Identify Change Point
print("\n" + "=" * 60)
print("CHANGE POINT IDENTIFICATION")
print("=" * 60)

# Get posterior distribution of tau
tau_samples = trace.posterior['tau'].values.flatten()
tau_mean = int(np.mean(tau_samples))
tau_median = int(np.median(tau_samples))
tau_mode = int(az.hdi(trace, var_names=["tau"])['tau'].values[0])

# Convert to dates
change_date_mean = returns_dates[tau_mean]
change_date_median = returns_dates[tau_median]
change_date_mode = returns_dates[tau_mode]

print(f"\nChange Point Statistics:")
print(f"  Mean index: {tau_mean} → Date: {change_date_mean.strftime('%Y-%m-%d')}")
print(f"  Median index: {tau_median} → Date: {change_date_median.strftime('%Y-%m-%d')}")
print(f"  Mode (HDI): {tau_mode} → Date: {change_date_mode.strftime('%Y-%m-%d')}")

# Plot posterior distribution of tau
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram of tau
axes[0].hist(tau_samples, bins=50, color='#2563eb', alpha=0.7, edgecolor='black')
axes[0].axvline(tau_mean, color='red', linestyle='--', linewidth=2, label=f'Mean: {tau_mean}')
axes[0].axvline(tau_median, color='green', linestyle='--', linewidth=2, label=f'Median: {tau_median}')
axes[0].set_title('Posterior Distribution of Change Point (τ)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time Index', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Posterior plot using arviz
az.plot_posterior(trace, var_names=["tau"], ax=axes[1], hdi_prob=0.95)
axes[1].set_title('Posterior Distribution with 95% HDI', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Use mean as the primary change point
primary_change_point = change_date_mean
print(f"\nPrimary Change Point: {primary_change_point.strftime('%Y-%m-%d')}")


In [ ]:
# Quantify Impact
print("\n" + "=" * 60)
print("QUANTIFYING IMPACT")
print("=" * 60)

# Get posterior means
mu1_samples = trace.posterior['mu_1'].values.flatten()
mu2_samples = trace.posterior['mu_2'].values.flatten()
sigma_samples = trace.posterior['sigma'].values.flatten()

mu1_mean = np.mean(mu1_samples)
mu2_mean = np.mean(mu2_samples)
sigma_mean = np.mean(sigma_samples)

# Calculate impact
impact = mu2_mean - mu1_mean
impact_pct = impact * 100

print(f"\nParameter Estimates:")
print(f"  μ₁ (before change): {mu1_mean:.6f}")
print(f"  μ₂ (after change): {mu2_mean:.6f}")
print(f"  σ (volatility): {sigma_mean:.6f}")
print(f"\nImpact:")
print(f"  Change in mean return: {impact:.6f}")
print(f"  Change in mean return (%): {impact_pct:.4f}%")

# Visualize parameter distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

az.plot_posterior(trace, var_names=["mu_1"], ax=axes[0], hdi_prob=0.95)
axes[0].set_title('Posterior: μ₁ (Before Change Point)', fontsize=12, fontweight='bold')

az.plot_posterior(trace, var_names=["mu_2"], ax=axes[1], hdi_prob=0.95)
axes[1].set_title('Posterior: μ₂ (After Change Point)', fontsize=12, fontweight='bold')

az.plot_posterior(trace, var_names=["sigma"], ax=axes[2], hdi_prob=0.95)
axes[2].set_title('Posterior: σ (Volatility)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Visualize change point on price series
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Plot prices with change point
ax1.plot(df.index, df['Price'], linewidth=1.5, color='#2563eb', alpha=0.8)
ax1.axvline(x=primary_change_point, color='red', linestyle='--', linewidth=2, 
            label=f'Change Point: {primary_change_point.strftime("%Y-%m-%d")}')
ax1.set_title('Brent Oil Prices with Detected Change Point', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price (USD per barrel)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot returns with change point
ax2.plot(returns_dates, returns, linewidth=0.8, color='#dc2626', alpha=0.6)
ax2.axvline(x=primary_change_point, color='red', linestyle='--', linewidth=2)
ax2.axhline(y=mu1_mean, color='blue', linestyle=':', linewidth=1.5, 
            label=f'μ₁ = {mu1_mean:.4f}')
ax2.axhline(y=mu2_mean, color='green', linestyle=':', linewidth=1.5, 
            label=f'μ₂ = {mu2_mean:.4f}')
ax2.set_title('Log Returns with Change Point and Mean Estimates', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Log Return', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save results
results = {
    'change_point_date': primary_change_point,
    'change_point_index': tau_mean,
    'mu_1': mu1_mean,
    'mu_2': mu2_mean,
    'sigma': sigma_mean,
    'impact': impact,
    'impact_pct': impact_pct
}

print("\n" + "=" * 60)
print("RESULTS SUMMARY")
print("=" * 60)
for key, value in results.items():
    if isinstance(value, pd.Timestamp):
        print(f"{key}: {value.strftime('%Y-%m-%d')}")
    else:
        print(f"{key}: {value}")
